In [7]:
import pandas as pd
import numpy as np
from datetime import *
import re
import csv
import pprint
import boto3
pp = pprint.PrettyPrinter(indent=2)
comprehend = boto3.client(service_name="comprehend", region_name="eu-west-1")

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/arbash-malik/ceutest/main/trip_advisor_resturaunts.csv')

In [3]:
df.head()

,hotel,location,ranking,rating,review_count,min_price_usd,max_price_usd,review,date
0,Bonoo Indian Tapas Restaurant,"675 Finchley Road, London NW2 2JP England","#1 of 15,800 Restaurants in London",5.0,"(2,673)",$25,$63,"Amazing spot! Accompanied with great staff, ve...",Date of visit: December 2023
1,Bonoo Indian Tapas Restaurant,"675 Finchley Road, London NW2 2JP England","#1 of 15,800 Restaurants in London",5.0,"(2,673)",$25,$63,Bonoo was suggested to us by some friends and ...,Date of visit: December 2023
2,Bonoo Indian Tapas Restaurant,"675 Finchley Road, London NW2 2JP England","#1 of 15,800 Restaurants in London",5.0,"(2,673)",$25,$63,Second time visiting and once again food was a...,Date of visit: December 2023
3,Bonoo Indian Tapas Restaurant,"675 Finchley Road, London NW2 2JP England","#1 of 15,800 Restaurants in London",5.0,"(2,673)",$25,$63,"Had a lovely meal at Bonoo, the chilli paneer ...",Date of visit: December 2023
4,Bonoo Indian Tapas Restaurant,"675 Finchley Road, London NW2 2JP England","#1 of 15,800 Restaurants in London",5.0,"(2,673)",$25,$63,"Hands down, best butter chicken ever. The staf...",Date of visit: December 2023


In [8]:
text = "wow, this is good"
pp.pprint(comprehend.detect_dominant_language(Text=text))

{ 'Languages': [{'LanguageCode': 'en', 'Score': 0.9982288479804993}],
  'ResponseMetadata': { 'HTTPHeaders': { 'content-length': '64',
                                         'content-type': 'application/x-amz-json-1.1',
                                         'date': 'Mon, 18 Dec 2023 13:07:13 '
                                                 'GMT',
                                         'x-amzn-requestid': 'dc3d6480-d6ab-4251-a828-79c22c42dc04'},
                        'HTTPStatusCode': 200,
                        'RequestId': 'dc3d6480-d6ab-4251-a828-79c22c42dc04',
                        'RetryAttempts': 0}}


In [9]:
text = "Hey, I'm feeling great today!"
print(comprehend.detect_sentiment(Text=text, LanguageCode="en"))

{'Sentiment': 'POSITIVE', 'SentimentScore': {'Positive': 0.9834643602371216, 'Negative': 0.0025514657609164715, 'Neutral': 0.012600112706422806, 'Mixed': 0.0013839955208823085}, 'ResponseMetadata': {'RequestId': '209bb831-504f-4e49-a30f-4fd7935dec62', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '209bb831-504f-4e49-a30f-4fd7935dec62', 'content-type': 'application/x-amz-json-1.1', 'content-length': '167', 'date': 'Mon, 18 Dec 2023 13:07:50 GMT'}, 'RetryAttempts': 0}}


In [20]:
# Importing helper libraries and setting helper variables
import pprint
import boto3
pp = pprint.PrettyPrinter(indent=2)
comprehend = boto3.client(service_name="comprehend", region_name="eu-west-1")


# Function for sentiment analysis
def run_sentiment_analysis_for_reviews(df):
    positive_scores = []
    neutral_scores = []
    negative_scores = []
    mixed_scores = []

    for index, row in df.iterrows():
        review_text = row['review']

        # Split the review text into chunks (less than 5000 characters)
        chunks = [review_text[i:i+4500] for i in range(0, len(review_text), 4500)]

        chunk_positive_scores = []
        chunk_neutral_scores = []
        chunk_negative_scores = []
        chunk_mixed_scores = []

        # Analyze sentiment for each chunk using AWS Comprehend
        for chunk in chunks:
            sentiment_result = comprehend.detect_sentiment(Text=chunk, LanguageCode="en")

            # Extract scores for each sentiment
            chunk_positive_scores.append(sentiment_result['SentimentScore']['Positive'])
            chunk_neutral_scores.append(sentiment_result['SentimentScore']['Neutral'])
            chunk_negative_scores.append(sentiment_result['SentimentScore']['Negative'])
            chunk_mixed_scores.append(sentiment_result['SentimentScore']['Mixed'])

        # This averaging was done to ensure each row had a differnt score.
        positive_scores.append(sum(chunk_positive_scores) / len(chunk_positive_scores))
        neutral_scores.append(sum(chunk_neutral_scores) / len(chunk_neutral_scores))
        negative_scores.append(sum(chunk_negative_scores) / len(chunk_negative_scores))
        mixed_scores.append(sum(chunk_mixed_scores) / len(chunk_mixed_scores))

    # Add new columns for sentiment scores to the DataFrame
    df['positive_score'] = positive_scores
    df['neutral_score'] = neutral_scores
    df['negative_score'] = negative_scores
    df['mixed_score'] = mixed_scores

In [21]:
run_sentiment_analysis_for_reviews(df)
df

,hotel,location,ranking,rating,review_count,min_price_usd,max_price_usd,review,date,sentiment,positive_score,neutral_score,negative_score,mixed_score
0,Bonoo Indian Tapas Restaurant,"675 Finchley Road, London NW2 2JP England","#1 of 15,800 Restaurants in London",5.0,"(2,673)",$25,$63,"Amazing spot! Accompanied with great staff, ve...",Date of visit: December 2023,POSITIVE,0.999664,0.000231,0.000021,0.000083
1,Bonoo Indian Tapas Restaurant,"675 Finchley Road, London NW2 2JP England","#1 of 15,800 Restaurants in London",5.0,"(2,673)",$25,$63,Bonoo was suggested to us by some friends and ...,Date of visit: December 2023,POSITIVE,0.999774,0.000164,0.000024,0.000038
2,Bonoo Indian Tapas Restaurant,"675 Finchley Road, London NW2 2JP England","#1 of 15,800 Restaurants in London",5.0,"(2,673)",$25,$63,Second time visiting and once again food was a...,Date of visit: December 2023,POSITIVE,0.999379,0.000349,0.000086,0.000186
3,Bonoo Indian Tapas Restaurant,"675 Finchley Road, London NW2 2JP England","#1 of 15,800 Restaurants in London",5.0,"(2,673)",$25,$63,"Had a lovely meal at Bonoo, the chilli paneer ...",Date of visit: December 2023,POSITIVE,0.999183,0.000705,0.000073,0.000039
4,Bonoo Indian Tapas Restaurant,"675 Finchley Road, London NW2 2JP England","#1 of 15,800 Restaurants in London",5.0,"(2,673)",$25,$63,"Hands down, best butter chicken ever. The staf...",Date of visit: December 2023,POSITIVE,0.999321,0.000600,0.000058,0.000021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104,Maple Casual Dining,"Vesterbrogade 24, Copenhagen 1620 Denmark","#5 of 1,992 Restaurants in Copenhagen",5.0,(759),NaN,NaN,We had a fantastic meal and the atmosphere was...,Date of visit: October 2023,POSITIVE,0.999861,0.000059,0.000031,0.000049
1105,Maple Casual Dining,"Vesterbrogade 24, Copenhagen 1620 Denmark","#5 of 1,992 Restaurants in Copenhagen",5.0,(759),NaN,NaN,What a delightful dinner we had at Maple Casua...,Date of visit: October 2023,POSITIVE,0.999786,0.000164,0.000019,0.000032
1106,Maple Casual Dining,"Vesterbrogade 24, Copenhagen 1620 Denmark","#5 of 1,992 Restaurants in Copenhagen",5.0,(759),NaN,NaN,We visited on our third night in Copenhagen an...,Date of visit: October 2023,POSITIVE,0.997937,0.000114,0.000316,0.001634
1107,Maple Casual Dining,"Vesterbrogade 24, Copenhagen 1620 Denmark","#5 of 1,992 Restaurants in Copenhagen",5.0,(759),NaN,NaN,"This is a great restaurant in every respect, w...",Date of visit: October 2023,MIXED,0.021919,0.000205,0.006052,0.971824


In [19]:
df['sentiment']

0       POSITIVE
1       POSITIVE
2       POSITIVE
3       POSITIVE
4       POSITIVE
          ...   
1104    POSITIVE
1105    POSITIVE
1106    POSITIVE
1107       MIXED
1108    POSITIVE
Name: sentiment, Length: 1109, dtype: object

In [23]:
df['ranking'] = df['ranking'].str.extract('(\d+)').astype(int)
df['review_count'] = df['review_count'].str.replace('(', '').str.replace(')', '').str.replace(',', '').astype(int)
df['min_price_usd'] = df['min_price_usd'].str.replace('$','').astype(float).round(0)
df['max_price_usd'] = df['max_price_usd'].str.replace('$','').astype(float).round(0)
df['date'] = df['date'].str.replace('Date of visit: ', '')

In [24]:
conditions = [
    df['location'].str.contains('London', case=False, na=False),
    df['location'].str.contains('Rome', case=False, na=False),
    df['location'].str.contains('Barcelona', case=False, na=False),
    df['location'].str.contains('Interlaken', case=False, na=False),
    df['location'].str.contains('Amsterdam', case=False, na=False),
    df['location'].str.contains('Venice', case=False, na=False),
    df['location'].str.contains('Prague', case=False, na=False),
    df['location'].str.contains('Budapest', case=False, na=False),
    df['location'].str.contains('Paris', case=False, na=False),
    df['location'].str.contains('Vienna', case=False, na=False),
    df['location'].str.contains('Brussels', case=False, na=False),
    df['location'].str.contains('Lisbon', case=False, na=False),
    df['location'].str.contains('Oslo', case=False, na=False),
    df['location'].str.contains('Stockholm', case=False, na=False),
    df['location'].str.contains('Copenhagen', case=False, na=False)    
]

values = ['London', 'Rome', 'Barcelona','Interlaken','Amsterdam','Venice','Prague','Budapest','Paris','Vienna','Brussels','Lisbon','Oslo','Stockholm','Copenhagen']


df['city'] = np.select(conditions, values, default=None)

In [25]:
conditions = [
    df['date'].str.contains('January', case=False, na=False),
    df['date'].str.contains('February', case=False, na=False),
    df['date'].str.contains('March', case=False, na=False),
    df['date'].str.contains('April', case=False, na=False),
    df['date'].str.contains('May', case=False, na=False),
    df['date'].str.contains('June', case=False, na=False),
    df['date'].str.contains('July', case=False, na=False),
    df['date'].str.contains('August', case=False, na=False),
    df['date'].str.contains('September', case=False, na=False),
    df['date'].str.contains('October', case=False, na=False),    
    df['date'].str.contains('November', case=False, na=False),
    df['date'].str.contains('December', case=False, na=False)]

values = ['1','2','3','4','5','6','7','8','9','10','11','12']


df['month'] = np.select(conditions, values, default=None)

In [26]:
df['review_length'] = df['review'].str.len()

In [27]:
df.to_csv('trip_advisor_resturaunts_sent.csv', sep =',', index = False, encoding = 'utf-8',quoting=csv.QUOTE_ALL)

In [28]:
df

,hotel,location,ranking,rating,review_count,min_price_usd,max_price_usd,review,date,sentiment,positive_score,neutral_score,negative_score,mixed_score,city,month,review_length
0,Bonoo Indian Tapas Restaurant,"675 Finchley Road, London NW2 2JP England",1,5.0,2673,25.0,63.0,"Amazing spot! Accompanied with great staff, ve...",December 2023,POSITIVE,0.999664,0.000231,0.000021,0.000083,London,12,174
1,Bonoo Indian Tapas Restaurant,"675 Finchley Road, London NW2 2JP England",1,5.0,2673,25.0,63.0,Bonoo was suggested to us by some friends and ...,December 2023,POSITIVE,0.999774,0.000164,0.000024,0.000038,London,12,325
2,Bonoo Indian Tapas Restaurant,"675 Finchley Road, London NW2 2JP England",1,5.0,2673,25.0,63.0,Second time visiting and once again food was a...,December 2023,POSITIVE,0.999379,0.000349,0.000086,0.000186,London,12,139
3,Bonoo Indian Tapas Restaurant,"675 Finchley Road, London NW2 2JP England",1,5.0,2673,25.0,63.0,"Had a lovely meal at Bonoo, the chilli paneer ...",December 2023,POSITIVE,0.999183,0.000705,0.000073,0.000039,London,12,143
4,Bonoo Indian Tapas Restaurant,"675 Finchley Road, London NW2 2JP England",1,5.0,2673,25.0,63.0,"Hands down, best butter chicken ever. The staf...",December 2023,POSITIVE,0.999321,0.000600,0.000058,0.000021,London,12,117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104,Maple Casual Dining,"Vesterbrogade 24, Copenhagen 1620 Denmark",5,5.0,759,NaN,NaN,We had a fantastic meal and the atmosphere was...,October 2023,POSITIVE,0.999861,0.000059,0.000031,0.000049,Copenhagen,10,195
1105,Maple Casual Dining,"Vesterbrogade 24, Copenhagen 1620 Denmark",5,5.0,759,NaN,NaN,What a delightful dinner we had at Maple Casua...,October 2023,POSITIVE,0.999786,0.000164,0.000019,0.000032,Copenhagen,10,219
1106,Maple Casual Dining,"Vesterbrogade 24, Copenhagen 1620 Denmark",5,5.0,759,NaN,NaN,We visited on our third night in Copenhagen an...,October 2023,POSITIVE,0.997937,0.000114,0.000316,0.001634,Copenhagen,10,811
1107,Maple Casual Dining,"Vesterbrogade 24, Copenhagen 1620 Denmark",5,5.0,759,NaN,NaN,"This is a great restaurant in every respect, w...",October 2023,MIXED,0.021919,0.000205,0.006052,0.971824,Copenhagen,10,427
